# PyRepo-Pal Database Design

This notebook outlines the database schema for PyRepo-Pal and includes scripts for initializing the database (initially with SQLite).

## Database Configuration

In [ ]:
from pathlib import Path
import sqlite3

# Define the database path (relative to the project root's src/data/ directory)
PROJECT_ROOT = Path().resolve().parent.parent # Assuming this notebook is in src/data/notebooks
DB_DIR = PROJECT_ROOT / "src" / "data" / "database"
DB_FILE = DB_DIR / "pyrepopal.db"

def ensure_db_directory():
    DB_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Database directory ensured: {DB_DIR}")

ensure_db_directory()
print(f"Database file will be located at: {DB_FILE}")

## Table Schemas

This section defines the conceptual schema for each table and provides the SQLite `CREATE TABLE` statements.

### 1. `analysis_sessions` Table

**Purpose:** To track each unique analysis run performed by PyRepo-Pal.

**Conceptual Attributes:**

| Column Name                   | General Data Type | SQLite Data Type | Constraints                               | Description                                                                 |
|-------------------------------|-------------------|------------------|-------------------------------------------|-----------------------------------------------------------------------------|
| `session_id`                  | Integer           | INTEGER          | PRIMARY KEY, AUTOINCREMENT                | Unique identifier for the session.                                          |
| `target_repository_identifier`| Text              | TEXT             | NOT NULL                                  | URL or local path of the repository being analyzed.                         |
| `analysis_timestamp`          | DateTime/Text     | TEXT             | NOT NULL                                  | Timestamp of when the analysis was initiated (e.g., ISO 8601 format).     |
| `user_notes`                  | Text              | TEXT             |                                           | Optional notes by the user about this session.                              |
| `status`                      | Text              | TEXT             |                                           | e.g., "pending", "data_collected", "prompt_generated", "ai_analyzed", "completed", "failed". |


In [ ]:
CREATE_ANALYSIS_SESSIONS_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS analysis_sessions (
    session_id INTEGER PRIMARY KEY AUTOINCREMENT,
    target_repository_identifier TEXT NOT NULL,
    analysis_timestamp TEXT NOT NULL,
    user_notes TEXT,
    status TEXT
);
"""

print("SQL for analysis_sessions table defined.")

### 2. `system_profiles` Table

**Purpose:** To store the user's system hardware/software profile at the time of a specific analysis session.

**Conceptual Attributes:**

| Column Name         | General Data Type | SQLite Data Type | Constraints                               | Description                                                                 |
|---------------------|-------------------|------------------|-------------------------------------------|-----------------------------------------------------------------------------|
| `profile_id`        | Integer           | INTEGER          | PRIMARY KEY, AUTOINCREMENT                | Unique identifier for the profile.                                          |
| `session_id`        | Integer           | INTEGER          | NOT NULL, FOREIGN KEY (analysis_sessions) | Links to the analysis session.                                              |
| `profile_timestamp` | DateTime/Text     | TEXT             | NOT NULL                                  | Timestamp when the profile was captured.                                    |
| `os_info`           | JSON/Text         | TEXT             |                                           | JSON string containing OS details.                                          |
| `cpu_info`          | JSON/Text         | TEXT             |                                           | JSON string containing CPU details.                                         |
| `ram_info`          | JSON/Text         | TEXT             |                                           | JSON string containing RAM details.                                         |
| `gpu_info`          | JSON/Text         | TEXT             |                                           | JSON string containing GPU details.                                         |
| `disk_info`         | JSON/Text         | TEXT             |                                           | JSON string containing disk details.                                        |
| `python_info`       | JSON/Text         | TEXT             |                                           | JSON string containing Python environment details.                          |


In [ ]:
CREATE_SYSTEM_PROFILES_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS system_profiles (
    profile_id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id INTEGER NOT NULL,
    profile_timestamp TEXT NOT NULL,
    os_info TEXT,
    cpu_info TEXT,
    ram_info TEXT,
    gpu_info TEXT,
    disk_info TEXT,
    python_info TEXT,
    FOREIGN KEY (session_id) REFERENCES analysis_sessions (session_id)
);
"""

print("SQL for system_profiles table defined.")

### 3. `repository_snapshots` Table

**Purpose:** To store the content of key files from the target repository at the time of a specific analysis session.

**Conceptual Attributes:**

| Column Name                       | General Data Type | SQLite Data Type | Constraints                               | Description                                                                 |
|-----------------------------------|-------------------|------------------|-------------------------------------------|-----------------------------------------------------------------------------|
| `snapshot_id`                     | Integer           | INTEGER          | PRIMARY KEY, AUTOINCREMENT                | Unique identifier for the snapshot.                                         |
| `session_id`                      | Integer           | INTEGER          | NOT NULL, FOREIGN KEY (analysis_sessions) | Links to the analysis session.                                              |
| `readme_content`                  | Text              | TEXT             |                                           | Content of README.md.                                                       |
| `requirements_txt_content`        | Text              | TEXT             |                                           | Content of requirements.txt.                                                |
| `environment_yaml_content`        | Text              | TEXT             |                                           | Content of environment.yaml.                                                |
| `existing_min_sys_reqs_content`   | Text              | TEXT             |                                           | Content of any found existing requirements file.                            |


In [ ]:
CREATE_REPOSITORY_SNAPSHOTS_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS repository_snapshots (
    snapshot_id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id INTEGER NOT NULL,
    readme_content TEXT,
    requirements_txt_content TEXT,
    environment_yaml_content TEXT,
    existing_min_sys_reqs_content TEXT,
    FOREIGN KEY (session_id) REFERENCES analysis_sessions (session_id)
);
"""

print("SQL for repository_snapshots table defined.")

### Database Initialization Script

The following cell contains a Python script to connect to the SQLite database and create the defined tables if they don't already exist.

In [ ]:
def initialize_database():
    ensure_db_directory() # Make sure directory exists
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        print(f"Connected to database: {DB_FILE}")
        
        table_creation_scripts = {
            "analysis_sessions": CREATE_ANALYSIS_SESSIONS_TABLE_SQL,
            "system_profiles": CREATE_SYSTEM_PROFILES_TABLE_SQL,
            "repository_snapshots": CREATE_REPOSITORY_SNAPSHOTS_TABLE_SQL,
            "generated_prompts": CREATE_GENERATED_PROMPTS_TABLE_SQL, # Added
            "ai_analysis_results": CREATE_AI_ANALYSIS_RESULTS_TABLE_SQL  # Added
        }
        
        for table_name, script in table_creation_scripts.items():
            print(f"Ensuring {table_name} table...")
            cursor.execute(script)
        
        conn.commit()
        print("Database initialized successfully with all tables.")
        
    except sqlite3.Error as e:
        print(f"SQLite error during database initialization: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

# To run the initialization:
# initialize_database()

To initialize or update the database schema, uncomment and run the `initialize_database()` call in the cell above.

### 4. `generated_prompts` Table

**Purpose:** To store the actual prompt text generated and sent to the AI for a session.

**Conceptual Attributes:**

| Column Name           | General Data Type | SQLite Data Type | Constraints                               | Description                                                                 |
|-----------------------|-------------------|------------------|-------------------------------------------|-----------------------------------------------------------------------------|
| `prompt_id`           | Integer           | INTEGER          | PRIMARY KEY, AUTOINCREMENT                | Unique identifier for the prompt.                                           |
| `session_id`          | Integer           | INTEGER          | NOT NULL, FOREIGN KEY (analysis_sessions) | Links to the analysis session.                                              |
| `prompt_type`         | Text              | TEXT             |                                           | e.g., "min_sys_reqs_determination", "dependency_resolution_cuda".       |
| `template_name_used`  | Text              | TEXT             |                                           | Name of the template file used.                                             |
| `prompt_content`      | Text              | TEXT             | NOT NULL                                  | The full text of the generated prompt.                                      |
| `creation_timestamp`  | DateTime/Text     | TEXT             | NOT NULL                                  | Timestamp when the prompt was created.                                      |


In [ ]:
CREATE_GENERATED_PROMPTS_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS generated_prompts (
    prompt_id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id INTEGER NOT NULL,
    prompt_type TEXT,
    template_name_used TEXT,
    prompt_content TEXT NOT NULL,
    creation_timestamp TEXT NOT NULL,
    FOREIGN KEY (session_id) REFERENCES analysis_sessions (session_id)
);
"""
print("SQL for generated_prompts table defined.")

### 5. `ai_analysis_results` Table

**Purpose:** To store the response from the AI.

**Conceptual Attributes:**

| Column Name                       | General Data Type | SQLite Data Type | Constraints                               | Description                                                                 |
|-----------------------------------|-------------------|------------------|-------------------------------------------|-----------------------------------------------------------------------------|
| `result_id`                       | Integer           | INTEGER          | PRIMARY KEY, AUTOINCREMENT                | Unique identifier for the result.                                           |
| `prompt_id`                       | Integer           | INTEGER          | NOT NULL, FOREIGN KEY (generated_prompts) | Links to the generated prompt.                                              |
| `ai_response_raw`                 | Text              | TEXT             | NOT NULL                                  | The raw response from the AI.                                               |
| `parsed_system_requirements_json` | JSON/Text         | TEXT             |                                           | Parsed JSON output for system requirements.                                 |
| `parsed_dependencies_json`        | JSON/Text         | TEXT             |                                           | Parsed JSON/structured output for dependencies.                             |
| `response_timestamp`              | DateTime/Text     | TEXT             | NOT NULL                                  | Timestamp when the AI response was received.                                |


In [ ]:
CREATE_AI_ANALYSIS_RESULTS_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS ai_analysis_results (
    result_id INTEGER PRIMARY KEY AUTOINCREMENT,
    prompt_id INTEGER NOT NULL,
    ai_response_raw TEXT NOT NULL,
    parsed_system_requirements_json TEXT,
    parsed_dependencies_json TEXT,
    response_timestamp TEXT NOT NULL,
    FOREIGN KEY (prompt_id) REFERENCES generated_prompts (prompt_id)
);
"""
print("SQL for ai_analysis_results table defined.")

In [ ]:
# Updated initialize_database function
def initialize_database():
    ensure_db_directory() # Make sure directory exists
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        print(f"Connected to database: {DB_FILE}")
        
        table_creation_scripts = {
            "analysis_sessions": CREATE_ANALYSIS_SESSIONS_TABLE_SQL,
            "system_profiles": CREATE_SYSTEM_PROFILES_TABLE_SQL,
            "repository_snapshots": CREATE_REPOSITORY_SNAPSHOTS_TABLE_SQL,
            "generated_prompts": CREATE_GENERATED_PROMPTS_TABLE_SQL,
            "ai_analysis_results": CREATE_AI_ANALYSIS_RESULTS_TABLE_SQL
        }
        
        for table_name, script in table_creation_scripts.items():
            print(f"Ensuring {table_name} table...") # This print was part of the loop already
            cursor.execute(script)
        
        conn.commit()
        print("Database initialized successfully with all tables.")
        
    except sqlite3.Error as e:
        print(f"SQLite error during database initialization: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

# To run the initialization:
# initialize_database()